# 📘 coleta_dados
### **Objetivo:**
- Coletar lista de deputados e seus eventos via API da Câmara

In [1]:
import os, time, json, csv, requests
from datetime import date

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
DIR_SAIDA = "../data/processed"
ARQ_DEPUTADOS = f"{DIR_SAIDA}/deputados.json"
ARQ_FREQ = f"{DIR_SAIDA}/freq_eventos.csv"

os.makedirs(DIR_SAIDA, exist_ok=True)

# janela temporal
DATA_INICIO = "2020-01-01"
DATA_FIM = str(date.today())  # hoje

# sessão HTTP enxuta
sessao = requests.Session()
sessao.headers.update({
    "Accept": "application/json",
    "User-Agent": "PUCRS-Assiduidade/1.0 (contato: seu_email@exemplo.com)"
})


### 1️⃣ Deputados:
- Geramos **deputados.json** - id, uri, nome, siglaPartido, uriPartido, siglaUF, idLegislatura, urlFoto e email


In [7]:
deputados, pagina = [], 1

while True:
    r = sessao.get(f"{BASE_URL}/deputados", params={"itens": 100, "pagina": pagina}, timeout=30)
    if r.status_code != 200:
        print(f"Falha em /deputados pág {pagina}: HTTP {r.status_code}")
        break

    lote = r.json().get("dados", [])
    if not lote:
        break

    deputados.extend(lote)
    pagina += 1
    time.sleep(0.08)  # polidez

with open(ARQ_DEPUTADOS, "w", encoding="utf-8") as f:
    json.dump(deputados, f, ensure_ascii=False, indent=2)

print(f"✅ Deputados coletados: {len(deputados)} → {ARQ_DEPUTADOS}")


✅ Deputados coletados: 513 → ../data/processed/deputados.json


### 2️⃣ Frequencia Eventos:
- Geramos **freq_eventos.csv** - id_deputado e num_eventos


In [8]:
# carregar ids dos deputados (sem pandas)
with open(ARQ_DEPUTADOS, "r", encoding="utf-8") as f:
    ids_deputados = [int(d["id"]) for d in json.load(f)]

# CSV de saída
with open(ARQ_FREQ, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_deputado", "num_eventos"])

total_eventos = 0
for i, dep_id in enumerate(ids_deputados, 1):
    total_dep = 0
    pagina = 1

    while True:
        params = {
            "dataInicio": DATA_INICIO,
            "dataFim": DATA_FIM,
            "itens": 100,
            "ordenarPor": "dataHoraInicio",
            "ordem": "ASC",
            "pagina": pagina,
        }
        r = sessao.get(f"{BASE_URL}/deputados/{dep_id}/eventos", params=params, timeout=30)
        if r.status_code != 200:
            # erro pontual → segue próximo deputado
            print(f"Dep {dep_id} pág {pagina}: HTTP {r.status_code} (pulando)")
            break

        dados = r.json().get("dados", [])
        if not dados:
            break

        total_dep += len(dados)
        pagina += 1
        time.sleep(0.03)  # polidez leve entre páginas

    # grava a contagem do deputado
    with open(ARQ_FREQ, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([dep_id, total_dep])

    total_eventos += total_dep
    # log simples a cada 25 deputados
    if i % 25 == 0 or i == len(ids_deputados):
        print(f"{i}/{len(ids_deputados)} deputados processados… acumulado eventos = {total_eventos}")

print("✅ Contagens geradas →", ARQ_FREQ)
print("📊 Total estimado de eventos (2020→hoje):", total_eventos)


25/513 deputados processados… acumulado eventos = 22791
50/513 deputados processados… acumulado eventos = 43357
75/513 deputados processados… acumulado eventos = 73656
100/513 deputados processados… acumulado eventos = 98562
125/513 deputados processados… acumulado eventos = 119016
150/513 deputados processados… acumulado eventos = 140364
175/513 deputados processados… acumulado eventos = 163224
200/513 deputados processados… acumulado eventos = 186869
225/513 deputados processados… acumulado eventos = 209426
250/513 deputados processados… acumulado eventos = 232206
275/513 deputados processados… acumulado eventos = 255617
300/513 deputados processados… acumulado eventos = 279538
325/513 deputados processados… acumulado eventos = 300883
350/513 deputados processados… acumulado eventos = 322198
375/513 deputados processados… acumulado eventos = 343082
400/513 deputados processados… acumulado eventos = 365972
425/513 deputados processados… acumulado eventos = 388985
450/513 deputados pro

### 3️⃣ Votos deputados:
- Geramos **votos_deputados.csv** - id_votacao e id_deputado




In [ ]:
# CÉLULA — Coleta simples de votos (2020–2025) via API REST, usando seus eventos_sessoes.csv

import csv, time
import pandas as pd

ARQ_EVENTOS_SESSOES = f"{DIR_SAIDA}/eventos_sessoes.csv"
ARQ_VOTOS = f"{DIR_SAIDA}/votos_deputados.csv"

# 1) carregar eventos e limitar a 2020–2025 (pelas datas do arquivo)
ev = pd.read_csv(ARQ_EVENTOS_SESSOES, dtype={"id_evento":"Int64","dataHoraInicio":"string","dataHoraFim":"string"})
ev["dataHoraInicio"] = pd.to_datetime(ev["dataHoraInicio"], errors="coerce")
ev = ev.dropna(subset=["dataHoraInicio"])
mask = (ev["dataHoraInicio"].dt.year >= 2020) & (ev["dataHoraInicio"].dt.year <= 2025)
eventos_ids = ev.loc[mask, "id_evento"].dropna().astype("int64").tolist()

# 2) CSV de saída (sem filtrar por deputado — mais simples e completo)
with open(ARQ_VOTOS, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_votacao","id_deputado","tipo_voto","dataHoraRegistro"])

def _first(d, *keys):
    """pega a primeira chave existente em d (dict); retorna None se não achar"""
    for k in keys:
        if isinstance(d, dict) and k in d:
            return d[k]
    return None

def _deep(d, path):
    """acessa caminho aninhado 'a.b.c' em dict"""
    cur = d
    for k in path.split("."):
        if isinstance(cur, dict) and k in cur:
            cur = cur[k]
        else:
            return None
    return cur

tot_votacoes = tot_votos = 0

for idx, ev_id in enumerate(eventos_ids, 1):
    # 3) votações do evento
    rv = sessao.get(f"{BASE_URL}/eventos/{ev_id}/votacoes", timeout=30)
    if rv.status_code != 200:
        # nem todo evento tem votação; segue
        continue
    votacoes = rv.json().get("dados", []) or []
    if not votacoes:
        continue

    for v in votacoes:
        # id da votação (geralmente 'id')
        id_vot = _first(v, "id", "idVotacao")
        tot_votacoes += 1

        # 4) votos nominais da votação
        rvotos = sessao.get(f"{BASE_URL}/votacoes/{id_vot}/votos", timeout=30)
        if rvotos.status_code != 200:
            continue
        votos = rvotos.json().get("dados", []) or []
        if not votos:
            continue

        with open(ARQ_VOTOS, "a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            for row in votos:
                # id_deputado pode vir em 'deputado_' ou 'deputado' ou 'idDeputado'
                dep_id = (
                    _first(row, "idDeputado")
                    or _deep(row, "deputado_.id")
                    or _deep(row, "deputado.id")
                )
                # tipo de voto (SIM/NÃO/ABSTENÇÃO/OBSTRUÇÃO/etc.) costuma estar em 'tipoVoto'
                tipo = _first(row, "tipoVoto") or _deep(row, "voto.tipoVoto")
                # timestamp: mais comum 'dataRegistroVoto'; fallbacks mantidos
                ts = (
                    _first(row, "dataRegistroVoto", "dataHoraRegistro", "dataHora")
                    or _deep(row, "voto.dataRegistroVoto")
                    or _deep(row, "voto.dataHoraRegistro")
                    or _deep(row, "voto.dataHora")
                )

                w.writerow([id_vot, dep_id, tipo, ts])
                tot_votos += 1

        time.sleep(0.015)  # polidez por votação

    if idx % 200 == 0:
        print(f"… {idx}/{len(eventos_ids)} eventos processados; votações até agora: {tot_votacoes}")

    time.sleep(0.01)  # polidez por evento

print(f"✅ Votações coletadas: {tot_votacoes}")
print(f"✅ Votos nominais gravados: {tot_votos}")
print(f"→ {ARQ_VOTOS}")


… 200/14319 eventos processados; votações até agora: 298


In [5]:
import pandas as pd
v = pd.read_csv("../data/processed/votos_deputados.csv", dtype={"id_votacao":"string","id_deputado":"Int64","tipo_voto":"string","dataHoraRegistro":"string"})
print(v.dtypes)
print(v["tipo_voto"].notna().mean(), v["dataHoraRegistro"].notna().mean())
v.head()


id_votacao          string[python]
id_deputado                  Int64
tipo_voto           string[python]
dataHoraRegistro    string[python]
dtype: object
0.0 0.0


,id_votacao,id_deputado,tipo_voto,dataHoraRegistro
0,1198512-250,204379,<NA>,<NA>
1,1198512-250,204560,<NA>,<NA>
2,1198512-250,204528,<NA>,<NA>
3,1198512-250,121948,<NA>,<NA>
4,1198512-250,74646,<NA>,<NA>


In [ ]:
# CÉL 1 (corrigida) — BAIXA VOTOS NOMINAIS POR ANO (ARQUIVOS) c/ normalização de IDs
import csv, json, time, datetime

ARQ_VOTOS = f"{DIR_SAIDA}/votos_deputados.csv"

# ids dos deputados (int)
with open(ARQ_DEPUTADOS, "r", encoding="utf-8") as f:
    deputados_json = json.load(f)
ids_int = {int(d["id"]) for d in deputados_json}
ids_str = {str(d["id"]) for d in deputados_json}  # para comparação robusta

def to_int_or_none(x):
    try:
        return int(str(x))
    except Exception:
        return None

with open(ARQ_VOTOS, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_votacao","id_deputado","tipo_voto","dataHoraRegistro"])

ano_ini = int(DATA_INICIO[:4])
ano_fim = datetime.date.today().year

total_reg_arq = total_votos_filtrados = 0
exemplos = 0  # para amostrar alguns matches

for ano in range(ano_ini, ano_fim + 1):
    url = f"https://dadosabertos.camara.leg.br/arquivos/votacoesVotos/json/votacoesVotos-{ano}.json"
    try:
        r = sessao.get(url, timeout=60)
        if r.status_code != 200:
            print(f"⚠️ {ano}: HTTP {r.status_code} em {url} (pulando)")
            continue

        payload = r.json()
        linhas = payload.get("dados", payload if isinstance(payload, list) else [])
        if not linhas:
            print(f"ℹ️ {ano}: arquivo sem linhas.")
            continue

        wrote_this_year = 0
        with open(ARQ_VOTOS, "a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            for row in linhas:
                # pegar id do deputado de forma robusta
                dep_id_raw = (
                    row.get("idDeputado") or
                    (row.get("deputado") or {}).get("id") or
                    (row.get("deputado_") or {}).get("id")
                )
                dep_id_str = str(dep_id_raw) if dep_id_raw is not None else None
                dep_id_int = to_int_or_none(dep_id_raw)

                # só grava se o deputado está na sua lista (comparar como str OU int)
                if dep_id_str in ids_str or (dep_id_int is not None and dep_id_int in ids_int):
                    w.writerow([
                        row.get("idVotacao") or row.get("id"),
                        dep_id_int if dep_id_int is not None else dep_id_str,
                        row.get("tipoVoto"),
                        row.get("dataHoraRegistro") or row.get("dataHora"),
                    ])
                    total_votos_filtrados += 1
                    wrote_this_year += 1

                    # amostra 3 exemplos para ver no log
                    if exemplos < 3:
                        print("exemplo:", {
                            "ano": ano,
                            "idVotacao": row.get("idVotacao") or row.get("id"),
                            "dep_id_raw": dep_id_raw,
                            "tipoVoto": row.get("tipoVoto")
                        })
                        exemplos += 1

        total_reg_arq += len(linhas)
        print(f"✅ {ano}: registros no arquivo = {len(linhas)}; filtrados este ano = {wrote_this_year}; acumulado = {total_votos_filtrados}")
        time.sleep(0.06)
    except Exception as e:
        print(f"⚠️ {ano}: erro processando {url} → {e}")

print(f"→ {ARQ_VOTOS}")


exemplo: {'ano': 2020, 'idVotacao': '1198512-250', 'dep_id_raw': '204379', 'tipoVoto': None}
exemplo: {'ano': 2020, 'idVotacao': '1198512-250', 'dep_id_raw': '204560', 'tipoVoto': None}
exemplo: {'ano': 2020, 'idVotacao': '1198512-250', 'dep_id_raw': '204528', 'tipoVoto': None}
✅ 2020: registros no arquivo = 159717; filtrados este ano = 82709; acumulado = 82709


### Situação parlamentar:

In [11]:
import os, time, json, csv, requests
from datetime import date

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
DIR_SAIDA = "../data/processed"
ARQ_DEPUTADOS = f"{DIR_SAIDA}/deputados.json"
ARQ_FREQ = f"{DIR_SAIDA}/freq_eventos.csv"

os.makedirs(DIR_SAIDA, exist_ok=True)

# janela temporal
DATA_INICIO = "2020-01-01"
DATA_FIM = str(date.today())  # hoje

# sessão HTTP enxuta
sessao = requests.Session()
sessao.headers.update({
    "Accept": "application/json",
    "User-Agent": "PUCRS-Assiduidade/1.0 (contato: seu_email@exemplo.com)"
})


r = sessao.get(f"{BASE_URL}/referencias/deputados/codSituacao")
situacoes = r.json()["dados"]
print(json.dumps(situacoes, indent=2, ensure_ascii=False))


[
  {
    "cod": "",
    "sigla": "A",
    "nome": "Afastado",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "C",
    "nome": "Convocado",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "E",
    "nome": "Exercício",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "F",
    "nome": "Fim de Mandato",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "L",
    "nome": "Licença",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "S",
    "nome": "Suplência",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "U",
    "nome": "Suspenso",
    "descricao": ""
  },
  {
    "cod": "",
    "sigla": "V",
    "nome": "Vacância",
    "descricao": ""
  }
]


### Ocupações por ID
- Geramos **ocupacoes.csv** - id_deputado, titulo


In [10]:
import csv
import json
import os, time, json, csv, requests
from datetime import date

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
DIR_SAIDA = "../data/processed"
ARQ_DEPUTADOS = f"{DIR_SAIDA}/deputados.json"
ARQ_FREQ = f"{DIR_SAIDA}/freq_eventos.csv"

os.makedirs(DIR_SAIDA, exist_ok=True)

# janela temporal
DATA_INICIO = "2020-01-01"
DATA_FIM = str(date.today())  # hoje

# sessão HTTP enxuta
sessao = requests.Session()
sessao.headers.update({
    "Accept": "application/json",
    "User-Agent": "PUCRS-Assiduidade/1.0 (contato: seu_email@exemplo.com)"
})

ARQ_DEPUTADOS = f"{DIR_SAIDA}/deputados.json"
with open(ARQ_DEPUTADOS, "r", encoding="utf-8") as f:
    ids_deputados = [int(d["id"]) for d in json.load(f)]

DIR_SAIDA = "../data/processed"
ARQ_OCUPACOES = f"{DIR_SAIDA}/ocupacoes.csv"
with open(ARQ_OCUPACOES, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_deputado", "titulo", "dataInicio", "dataFim"])

for dep_id in ids_deputados:
    r = sessao.get(f"{BASE_URL}/deputados/{dep_id}/ocupacoes", timeout=30)
    if r.status_code != 200:
        continue
    for o in r.json()["dados"]:
        csv.writer(open(ARQ_OCUPACOES, "a", newline="", encoding="utf-8")).writerow([
            dep_id,
            o.get("titulo"),
            o.get("dataInicio"),
            o.get("dataFim")
        ])


### Eventos Sessões:
- Geramos **eventos_sessoes.csv** - id_evento, dataHoraInicio e dataHoraFim


In [ ]:
# CÉL A — SESSÕES + PRESENÇAS
import csv, json, time

ARQ_EVENTOS_SESSOES = f"{DIR_SAIDA}/eventos_sessoes.csv"
ARQ_PRESENCAS = f"{DIR_SAIDA}/presencas_detalhe.csv"

# ids-alvo (seus deputados)
with open(ARQ_DEPUTADOS, "r", encoding="utf-8") as f:
    ids_deputados_set = {int(d["id"]) for d in json.load(f)}

# descobrir códigos de "Sessão Deliberativa"
ref = sessao.get(f"{BASE_URL}/referencias/eventos/codTipoEvento", timeout=30)
ref.raise_for_status()
tipos = ref.json().get("dados", [])
codigos_sessao = {
    t["id"] for t in tipos
    if "sessão deliberativa" in (t.get("descricao","").lower())
}
if not codigos_sessao:
    print("⚠️ Não achei códigos de 'Sessão Deliberativa' nas referências; seguirei sem filtro por tipo.")

# listar eventos no período
with open(ARQ_EVENTOS_SESSOES, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_evento","dataHoraInicio","dataHoraFim","id_orgao","descricaoTipo"])

pagina, eventos_ids = 1, []
while True:
    params = {
        "dataInicio": DATA_INICIO,
        "dataFim": DATA_FIM,
        "itens": 100,
        "pagina": pagina,
        "ordem": "ASC",
        "ordenarPor": "dataHoraInicio",
    }
    if codigos_sessao:
        params["codTipoEvento"] = ",".join(str(x) for x in codigos_sessao)

    re = sessao.get(f"{BASE_URL}/eventos", params=params, timeout=30)
    if re.status_code != 200:
        print(f"⚠️ /eventos pág {pagina}: HTTP {re.status_code} (parando)")
        break
    bloco = re.json().get("dados", [])
    if not bloco:
        break

    with open(ARQ_EVENTOS_SESSOES, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        for ev in bloco:
            eventos_ids.append(ev.get("id"))
            w.writerow([
                ev.get("id"),
                ev.get("dataHoraInicio"),
                ev.get("dataHoraFim"),
                ev.get("idOrgao"),
                (ev.get("tipoEvento") or {}).get("descricao") if isinstance(ev.get("tipoEvento"), dict) else None
            ])

    pagina += 1
    time.sleep(0.03)

print(f"✅ Sessões coletadas: {len(eventos_ids)} → {ARQ_EVENTOS_SESSOES}")

# presenças por evento (opcional, útil p/ % presença)
with open(ARQ_PRESENCAS, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerow(["id_evento","id_deputado","tipo_presenca"])

total_presencas = 0
for idx, ev_id in enumerate(eventos_ids, 1):
    rp = sessao.get(f"{BASE_URL}/eventos/{ev_id}/presencas", timeout=30)
    if rp.status_code != 200:
        continue
    dados = rp.json().get("dados", [])
    if not dados:
        continue

    with open(ARQ_PRESENCAS, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        for p in dados:
            dep = p.get("deputado_", p.get("deputado", {}))
            dep_id = dep.get("id") if isinstance(dep, dict) else p.get("idDeputado")
            if dep_id in ids_deputados_set:
                w.writerow([ev_id, dep_id, p.get("tipoRegistro") or p.get("situacao") or p.get("presenca")])
                total_presencas += 1

    if idx % 200 == 0:
        print(f"… {idx}/{len(eventos_ids)} eventos processados")
    time.sleep(0.02)

print(f"✅ Presenças gravadas: {total_presencas} → {ARQ_PRESENCAS}")


⚠️ Não achei códigos de 'Sessão Deliberativa' nas referências; seguirei sem filtro por tipo.
✅ Sessões coletadas: 14319 → ../data/processed/eventos_sessoes.csv
